In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 实现基本的转置卷积运算
def trans_conv(X, K):
    h, w = K.shape  # 卷积核的宽、高
    Y = torch.zeros(
        (X.shape[0] + h - 1, X.shape[1] + w - 1))  # 正常的卷积后尺寸为(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i + h, j:j + w] += X[i, j] * K  # 按元素乘法，加回到自己矩阵
    return Y

In [3]:
# 验证上述实现输出
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [4]:
# 使用高级API获得相同的结果
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)  # 输入通道数为1，输出通道数为1
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [5]:
# 填充、步幅和多通道

# 填充在输出上，padding=1，之前输出3x3，现在上下左右都填充了1，那就剩下中心那个元素了
# 填充为1，就是把输出最外面的一圈当作填充
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [6]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [7]:
# 多通道
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

In [8]:
# 与矩阵变换的联系
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)  # 卷积
Y

tensor([[27., 37.],
        [57., 67.]])

In [9]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    print(k)
    print(W)
    print(K)
    k[:2], k[3:5] = K[0, :], K[1, :]
    print(k)
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

In [10]:
# 每一行向量表示在一个位置的卷积操作，0填充表示卷积核未覆盖到的区域。
# 输入大小为 3 * 3 的图片，拉长一维向量后变成 1 * 9 的向量
# 输入大小为 3 * 3 的图片，卷积核为 2 * 2，则输出图片为 2 * 2，拉长后变为 4 * 1 的向量
# kernel2matrix函数将卷积核改为稀疏矩阵C后矩阵情况
W = kernel2matrix(K)
W

tensor([0., 0., 0., 0., 0.])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([1., 2., 0., 3., 4.])


tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [11]:
print(X)
print(X.reshape(-1))
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])


tensor([[True, True],
        [True, True]])

In [12]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)  # 由卷积后的图像乘以转置卷积后，得到的并不是原图像，而是尺寸一样

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])